In [ ]:
#Questions
#determine how to slow down requests to craigslist
#how to structure data - many tables of one large dataframe

In [1]:
import requests # Helps construct the request to send to the API
import json # JSON helper functions
from bs4 import BeautifulSoup

In [104]:
#get list of craiglist listing urls

markets = [['washingtondc','nva']]

for market in markets:
    #create url for each specified market
    url = 'http://' + market[0] + '.craigslist.org/search/' + market[1] + '/apa'

    # Make the request
    response = requests.post(url)
    
    #place data in Beautiful soup object
    soup = BeautifulSoup(response.text)

    #create a list of all the listing ids - to create links to each listing page
    listing_ids = []
    data_pid = soup.find_all('p', class_='row')
    for listing_id in data_pid:
        listing_ids.append(listing_id['data-pid'])

In [100]:
#display the first five listing ids
listing_ids[0:5]

[u'4958213410', u'4935837515', u'4960324324', u'4964981993', u'4965024142']

In [115]:
# get data from craigslist

def get_craigslist_listing(id):
    url = 'http://' + market[0] + '.craigslist.org/' + market[1] + '/apa/' + id + '.html'
    
    # Make the request
    response = requests.post(url)

    # Confirm the response worked
    response.status_code
    
    #place data in Beautiful soup object
    soup = BeautifulSoup(response.text)

    return soup

In [117]:
#confirm get_craigslist_listing works
print(get_craigslist_listing(listing_ids[0]).prettify())

http://washingtondc.craigslist.org/nva/apa/4958213410.html
<!DOCTYPE html>
<html class="no-js">
 <head>
  <title>
   (DON'T MISS)HOUSE FOR RENT 2200/month
  </title>
  <meta content="NOARCHIVE,NOFOLLOW" name="robots">
   <meta content="A single family home for rent. It is in convenient location PW pkwy, I-95, close to public transportation. Close to Potomac mill and other shopping center. With six bedroom, 3 bathroom, kitchen,..." name="description">
    <meta content="preview" name="twitter:card">
     <meta content="A single family home for rent. It is in convenient location PW pkwy, I-95, close to public transportation. Close to Potomac mill and other shopping center. With six bedroom, 3 bathroom, kitchen,..." property="og:description">
      <meta content="http://images.craigslist.org/00D0D_fzO95yYKTID_600x450.jpg" property="og:image">
       <meta content="craigslist" property="og:site_name">
        <meta content="(DON'T MISS)HOUSE FOR RENT 2200/month" property="og:title">
      

In [121]:
for id in listing_ids:
    x = get_craigslist_listing(id)
    print x.find('title')

http://washingtondc.craigslist.org/nva/apa/4958213410.html
<title>(DON'T MISS)HOUSE FOR RENT 2200/month</title>
http://washingtondc.craigslist.org/nva/apa/4935837515.html
<title>Condo for rent in manassas, VA</title>
http://washingtondc.craigslist.org/nva/apa/4960324324.html
<title>Come check out all the perks, 1 mo free rent w/13 mo lease WOW!</title>
http://washingtondc.craigslist.org/nva/apa/4964981993.html
<title>Save $5K 4 IMMEDIATE MOVE IN@$1799+Waived Fees+Free Prkg=CALL NOW!!!</title>
http://washingtondc.craigslist.org/nva/apa/4965024142.html
<title>1 Bed and Den available Now! Near East Falls Church Metro</title>
http://washingtondc.craigslist.org/nva/apa/4965033473.html
<title>1 Br, 1 Ba newly renovated Condo</title>
http://washingtondc.craigslist.org/nva/apa/4951668084.html
<title>Comfort and Convenience 2B+den(3bath)- Ballston Metro - Near Schools</title>
http://washingtondc.craigslist.org/nva/apa/4961769063.html
<title>Spacious Duplex in Clarendon area</title>
http://washi

In [4]:
#get attributes about property - preset choices from craigslist
attribute_list = {}

x = soup.find('p',class_='attrgroup').find_all('span')
for attribute in x:

    if 'BR' in attribute.text:
        attribute_list['bedroom'] = attribute.text.split('/')[0].replace('BR','')
    if 'Ba' in attribute.text:
        attribute_list['bathroom'] = attribute.text.split('/')[1].replace('Ba','')
    if 'ft' in attribute.text:
        attribute_list['square_footage'] = attribute.text.replace('ft2','')
    if attribute.text in ['apartment', 'condo', 'cottage/cabin', 'duplex', 'flat', 
        'house', 'in-law', 'loft', 'townhouse','manufactured', 'assisted living', 'land']:
        attribute_list['housing_type'] = attribute.text
    if 'cat' in attribute.text:
        attribute_list['cat'] = attribute.text
    if 'dog' in attribute.text:
        attribute_list['dog'] = attribute.text
    if attribute.text in ['w/d in unit','laundry in bldg','laundry on site','w/d hookups']:
        attribute_list['laundry'] = attribute.text 
    if attribute.text in ['carport', 'attached garage', 'detached garage',  'off-street parking', 'street parking', 'valet parking']:
        attribute_list['parking'] = attribute.text
    if 'smoking' in attribute.text:
        attribute_list['smoking'] = attribute.text

attribute_list['availability'] = soup.find('span', class_='housing_movein_now property_date')['today_msg']
attribute_list['data_available'] = soup.find('span', class_='housing_movein_now property_date')['date']

attribute_list

{'availability': u'available now',
 'bathroom': u' 1',
 'bedroom': u'1 ',
 'cat': u'cats are OK - purrr',
 'data_available': u'2015-04-15',
 'dog': u'dogs are OK - wooof',
 'housing_type': u'apartment',
 'laundry': u'w/d in unit',
 'parking': u'attached garage',
 'smoking': u'no smoking',
 'square_footage': u'763'}

In [6]:
x = soup.find_all('h2')
for i in x:
    print i

<h2 class="postingtitle">
<span class="star"></span>
<span class="price">$2060</span> <span class="housing">/ 1br - 763ft<sup>2</sup> - </span>You Want To Live Here!<small> (Arlington, VA)</small>
</h2>


In [8]:
#get the listing price
x = soup.find('h2')
print x.contents[3].text

$2060


In [9]:
#get the number of bedrooms
x = soup.find('h2')
print x.contents[5].text.split('-')[0].replace('/','')

 1br 


In [10]:
#get the square footage of the listing
x = soup.find('h2')
print x.contents[5].text.split('-')[1][0:-2] #remove extra characters at start of string and '2' at end for square ft

 763ft


In [11]:
#Create a list of dicts containing image number, image link, and image size
image_data = []

images = soup.find('figure').find_all('div')
for pic in images[-1]:    
    pic_dict = {}
    pic_dict['image_number'] = pic['title']
    pic_dict['href'] = pic['href']
    pic_dict['image_size'] = pic['href'].split('_')[-1]
    
    image_data.append(pic_dict)

image_data

[{'href': u'http://images.craigslist.org/01515_dhEvaZdyFPU_600x450.jpg',
  'image_number': u'1',
  'image_size': u'600x450.jpg'},
 {'href': u'http://images.craigslist.org/00j0j_hq6pUcTET8_600x450.jpg',
  'image_number': u'2',
  'image_size': u'600x450.jpg'},
 {'href': u'http://images.craigslist.org/00505_eYldlDiVYGw_600x450.jpg',
  'image_number': u'3',
  'image_size': u'600x450.jpg'},
 {'href': u'http://images.craigslist.org/00F0F_40M7JupVWHP_600x450.jpg',
  'image_number': u'4',
  'image_size': u'600x450.jpg'}]

In [12]:
#create a dict with  location data: country, state, city, address, longitude, latitude, and location accuracy metric

#get longitude, latitude, and location accuracy metric
location = soup.find('div', class_='viewposting')
location_data_accuracy = soup.find('div', class_='viewposting')['data-accuracy']
latitude = soup.find('div', class_='viewposting')['data-latitude']
longitude = soup.find('div', class_='viewposting')['data-longitude']

#get country, state, city
map = soup.find('p', class_='mapaddress').find_all('a')[1]['href'] #get yahoo maps link - easier to extract data than google maps
country = map.split('country=')[1]
state = map.split('csz=')[1].split('&')[0].split('+')[1]
city  = map.split('csz=')[1].split('&')[0].split('+')[0]

#get address
address = soup.find('div', class_='mapaddress').text

#place all location data into dict
location_dict = {}
location_dict['location_data_accuracy'] = location_data_accuracy
location_dict['latitude'] = latitude
location_dict['longitude'] = longitude
location_dict['country'] = country
location_dict['state'] = state
location_dict['city'] = city
location_dict['address'] = address

location_dict

{'address': u'2250 Clarendon Blvd',
 'city': u'Arlington',
 'country': u'US',
 'latitude': u'38.890353',
 'location_data_accuracy': u'10',
 'longitude': u'-77.086406',
 'state': u'VA'}

In [13]:
#get the written user description of the listing
listing_description = soup.find('section', class_='userbody').find('section')

print listing_description.text


 CALL US TODAY! 
show contact info
COURTHOUSE PLAZA1 bedroom 1 bath   763sqft   $2,060 - $2,150  DESCRIPTION:  Simply perfect 1 bedroom with balcony! This apartment is perfect with a fully equipped kitchen great living room walk in closet sunny windows and even a balcony! Call today don't let this one get away!  Enjoy easy Metro access along with a rooftop deck, state-of-the-art fitness center and concierge service. Our convenient location on Clarendon Blvd means great shopping and dining at your front door.  2250 Clarendon Blvd  Arlington, va 22201  PROPERTY WEBSITE:  http://www.eqrworld.com/virginia/alexandria-arlington-apartments/courthouse/courthouse-plaza-apartments.aspx?ilsid=93

